In [1]:
import warnings
warnings.filterwarnings('ignore')

!nvidia-smi

Sat Sep 28 03:32:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
print(f"Torch + Cuda Version: {torch.__version__}")
print(f"Cuda Version Requied: {torch.version.cuda}")
print(f"Cuda Available: {torch.cuda.is_available()}")
print(f"Cuda Device Count: {torch.cuda.device_count()}")
# print(f"Cuda Device Name: {torch.cuda.get_device_name(0)}")
!python --version

Torch + Cuda Version: 2.4.1+cu121
Cuda Version Requied: 12.1
Cuda Available: True
Cuda Device Count: 1
Python 3.10.12


In [4]:
%cd /content/drive/MyDrive/MeshAnythingV2
%ls

/content/drive/MyDrive/MeshAnythingV2
adjacent_mesh_tokenization.py  gt_examples/   mesh_to_pc.py  point_cloud_inference.ipynb
app.py                         LICENSE.txt    npy_files/     __pycache__/
demo/                          main.py        output/        README.md
examples/                      MeshAnything/  pc_examples/   requirements.txt


In [5]:
CURRENCT_WORKING_DIR = %pwd
print(f"Current Working Directory Now: '{CURRENCT_WORKING_DIR}'")

Current Working Directory Now: '/content/drive/MyDrive/MeshAnythingV2'


In [6]:
BASE_DIR = "/content/drive/MyDrive/MeshAnythingV2/"
NPY_FILES_DIR = BASE_DIR + "npy_files/"

POINT_CLOUD_JSON = "/content/drive/MyDrive/Data/sample/data/08cf73ec-313d-436b-8ed9-c2b3034844ba/dsm.json"
# INPUT_POINT_CLOUD = "/content/drive/MyDrive/MeshAnythingV2/pc_examples/grenade.npy"
NPY_file = NPY_FILES_DIR + POINT_CLOUD_JSON.split("/")[-1].split(".")[0] + ".npy"

OUTPUT_DIR = "/content/drive/MyDrive/MeshAnythingV2/output"

In [7]:
!mkdir -p {NPY_FILES_DIR}

In [8]:
!mkdir -p {OUTPUT_DIR}

In [13]:
!pip install -r /content/drive/MyDrive/MeshAnythingV2/requirements.txt -q
!pip install flash-attn --no-build-isolation -q
!pip install open3d -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 110.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 85.1 MB/s eta 0:00:00


In [14]:
import json
import numpy as np
import open3d as o3d
from typing import Any

# Step 1: Load the JSON file containing the XYZ coordinates
def load_json(json_file) -> np.ndarray:
    with open(json_file, 'r') as f:
        data: Any = json.load(f)
    return np.array(data)  # Convert the list of lists into a NumPy array

# Step 2: Compute normals using Open3D
def compute_normals(xyz_points) -> np.ndarray:
    # Create an Open3D PointCloud object
    pcd: Any = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz_points)

    # Estimate the normals
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

    # Retrieve the normals
    normals: np.ndarray[Any, np.dtype[Any]] = np.asarray(pcd.normals)
    return normals

# Step 3: Save as .npy in the (N, 6) pc_normal format
def save_as_npy(xyz_points, normals, npy_file):
    # Combine the XYZ coordinates with their corresponding normals
    pc_normal_data = np.hstack((xyz_points, normals))

    # Save the array to an .npy file
    np.save(npy_file, pc_normal_data)
    print(f"Point cloud with normals saved to {npy_file}")


# Example usage
xyz_points = load_json(POINT_CLOUD_JSON)
normals = compute_normals(xyz_points)
save_as_npy(xyz_points, normals, NPY_file)

Point cloud with normals saved to /content/drive/MyDrive/MeshAnythingV2/npy_files/dsm.npy


In [15]:
# inference for single file
!python main.py --input_path {NPY_file} --out_dir {OUTPUT_DIR} --input_type pc_normal

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:463: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/content/drive/MyDrive/MeshAnythingV2/MeshAnything/miche/michelangelo/models/modules/checkpoint.py:41: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, run_function, length, *args):
/content/drive/MyDrive/MeshAnythingV2/MeshAnything/miche/michelangelo/models/modules/checkpoint.py:52: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, *output_grads):
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always 